## Human classification convulution learning

###checking if using GPU

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Not connected to a GPU
Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [2]:
import os
from typing import List, Tuple

import imageio
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.layers.core import (
    Activation,
    Reshape,
    Flatten,
    Dense,
)
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [3]:
%cd /content/drive/MyDrive/human_detection_project/Database/human_detection_dataset

/content/drive/MyDrive/human_detection_project/Database/human_detection_dataset


In [4]:
ls


data_0/  data_1/


In [5]:
IMAGE_SHAPE = (256, 256, 3)

##load images to people and no_people lists

In [6]:
people_path = "data_1/"
no_people_path = "data_0/"
people = []
no_people = []
range_t = [i for i in range(231)]

people_dir = os.listdir(people_path)
no_people_dir = os.listdir(no_people_path)

for i in tqdm(range_t):
    people.append(Image.open(f"{people_path}{people_dir[i]}"))

for i in tqdm(range_t):
    no_people.append(Image.open(f"{no_people_path}{no_people_dir[i]}"))

100%|██████████| 231/231 [01:06<00:00,  3.48it/s]


In [7]:
def reshape_images(images):
  """
  reshape the images that is not in shape: (256, 256, 3)
  """
  reshaped_images = []
  for i, img in enumerate(images):
      if img.size != (256, 256):
          img = img.resize((256, 256))
      new_img = np.asarray(img)
      new_img = new_img.astype(np.float32)
      if new_img.shape == IMAGE_SHAPE:
          reshaped_images.append(new_img)
  return reshaped_images

In [8]:
people_images = reshape_images(people)
no_people_images = reshape_images(no_people)

In [9]:
def check_shape(img_lst: list):
  for i, img in enumerate(img_lst):
    assert img.shape == IMAGE_SHAPE, f"error with image number {i}"

chack shape of the images

In [10]:
check_shape(people_images)
check_shape(no_people_images)

## lable the images
1 to people
0 to non people

In [11]:
shuffled_data = [(img, 1.0) for img in people_images]
shuffled_data.extend([(img, 0.0) for img in no_people_images])

In [12]:
# shuffle the images
random.shuffle(shuffled_data)

##normalize the images to rgb range [0-1]

In [13]:
norm = lambda mat: ((mat[0] - mat[0].min()) / mat[0].max(), mat[1])

shuffled_data = map(norm, shuffled_data)
shuffled_data = list(shuffled_data)

##define imaged and lable lists

In [14]:
images = []
labels = []

for img, label in shuffled_data:
  images.append(img)
  labels.append(label)

assert len(images) == len(labels), "labels and images should have the same length"

images = np.array(images)
labels = np.array(labels)

##data spliting

In [15]:
n = len(images)
split = int(n * 0.7)

train_images = images[: split]
train_labels = labels[: split]
test_images = images[split:]
test_labels = labels[split:]

# val_images = train_images[int(0.9*len(train_images)):]
# val_labels = train_labels[int(0.9*len(train_images)):]

n

411

#Define the Model

##custom model

In [16]:
def model_generator():
    IMAGE_SIZE = 256
    RGB = 3
    NFILTERS = 8
    CONV_PARAMS = {"kernel_size": 3, "use_bias": False, "padding": "same"}

    model = Sequential()
    # model.add(Reshape((64, IMAGE_SIZE, IMAGE_SIZE, RGB), input_shape=(IMAGE_SIZE, IMAGE_SIZE, RGB)))
    model.add(Reshape((IMAGE_SIZE, IMAGE_SIZE, RGB)))

    # adding the model layers
    for i in [1, 2, 4, 8, 16, 32, 64]:
        model.add(Conv2D(NFILTERS * i, **CONV_PARAMS))
        model.add(BatchNormalization(momentum=0.1))
        model.add(Activation("relu"))
        model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [17]:
def trainer(model: Sequential, images, labels, epochs=10, new_model=False):
    BATCH_SIZE = 64
    # if os.path.exists("model1.hdf5"):
    # return
    if new_model:
        model = keras.models.load_model("model1.hdf5")
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics='accuracy')  # adam = gradient descent algorithm
    model.fit(x=images, y=labels, batch_size=BATCH_SIZE, verbose=1, epochs=epochs, validation_split=0.1)
    model.save("model2.hdf5")

##evaluate the model and return the prediction

In [18]:
def eval(model, test_x: np.array, test_y: np.array) -> Tuple[np.array, float]:
    for layer in model.layers:
      if "batch_normalization" in layer.name:
        layer.trainable = False
        print("weigths:", len(layer.weights))
        print("trainable_weights:", len(layer.trainable_weights))
        print("non_trainable_weights:", len(layer.non_trainable_weights))
    # model.summary()
    threshhold = 0.5
    predicted_y = []
    for i, img in enumerate(test_x):
        pred = model.predict(img[None])
        pred = 1 if pred[0][0] > threshhold else 0
        predicted_y.append(pred)

    accuracy = sum([1 if predicted_y[i] == int(test_y[i]) else 0 for i in range(len(predicted_y))]) / len(predicted_y)
    with open("/content/drive/MyDrive/human_detection_project/results/classification_results.txt", "w") as f:
        f.write(f"this is my prediction {predicted_y}")
        f.write(f"this is True {test_y}")
        f.write(f"accuracy score: {accuracy}")
    return np.array(predicted_y), accuracy

In [19]:
def plot_images(images: List[np.ndarray],
                preds: np.ndarray,
                labels: np.ndarray,
                size: Tuple[int, int] = (4, 4)) -> None:
    f, axs = plt.subplots(size[0], size[1], figsize=(4 * size[0], 4 * size[1]))
    for img, p, label, ax in zip(images, preds, labels, axs.ravel()):
        ax.imshow(img)
        ax.set_title(f"Pred: {p}, Actual: {label}")
        ax.axis('off')
    plt.show("prediction.png")

In [20]:
%cd /content/drive/MyDrive/human_detection_project/models
%ls

/content/drive/MyDrive/human_detection_project/models
model1.hdf5  model2.hdf5


##Generate the model

In [21]:
# data sets:  train_images, train_labels, test_images, test_labels
# 291 train images, 126 test images

model = model_generator()
trainer(model, train_images, train_labels, epochs=15)

Epoch 1/15
5/5 [==============================] - 18s 3s/step - loss: 2.0867 - accuracy: 0.5388 - val_loss: 1.2544 - val_accuracy: 0.4828
Epoch 2/15
5/5 [==============================] - 16s 3s/step - loss: 0.7083 - accuracy: 0.6667 - val_loss: 1.1381 - val_accuracy: 0.4483
Epoch 3/15
5/5 [==============================] - 16s 3s/step - loss: 0.7012 - accuracy: 0.6628 - val_loss: 1.2955 - val_accuracy: 0.4828
Epoch 4/15
5/5 [==============================] - 16s 3s/step - loss: 0.6262 - accuracy: 0.6318 - val_loss: 0.7024 - val_accuracy: 0.5517
Epoch 5/15
5/5 [==============================] - 16s 3s/step - loss: 0.6439 - accuracy: 0.6085 - val_loss: 1.2088 - val_accuracy: 0.5517
Epoch 6/15
5/5 [==============================] - 16s 3s/step - loss: 0.5703 - accuracy: 0.7287 - val_loss: 1.1863 - val_accuracy: 0.4483
Epoch 7/15
5/5 [==============================] - 16s 3s/step - loss: 0.5535 - accuracy: 0.7209 - val_loss: 1.4684 - val_accuracy: 0.4828
Epoch 8/15
5/5 [==================

In [22]:
model_t = keras.models.load_model("model1.hdf5")
# prediction, accuracy = eval(model_t, test_images, test_labels)
prediction, accuracy = eval(model_t, test_images, test_labels)
# prediction, accuracy = eval(model_t, train_images, train_labels)

weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4


In [23]:
model_n = keras.models.load_model("model2.hdf5")
prediction, accuracy = eval(model_n, test_images, test_labels)


weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4
weigths: 4
trainable_weights: 0
non_trainable_weights: 4


In [24]:
# model = keras.models.load_model("model1.hdf5")
# results = model.evaluate(test_images, test_labels, batch_size=12)
# print("test loss, test acc:", results)
# predicted_lable, accuracy = eval(test_images, test_labels)
# plot_images(test_images, predicted_lable, test_labels)

# pred = model.predict(train_images[10][None])
# print('pred', pred)
# print('true', train_labels[10])